<a href="https://colab.research.google.com/github/Shri-Aakash/SDP-Tomato/blob/main/VGG_19_Custom_training_SDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Nov 10 09:52:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Lambda,Flatten,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
tf.config.list_physical_devices('GPU')
sys_details = tf.sysconfig.get_build_info()
cuda_version = sys_details["cuda_version"]
print(cuda_version)

11.2


In [4]:
import os

In [5]:
imgShape=[224,224,3]
batchSize=16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
dataPath='/content/drive/MyDrive/VGG-16 Tomato'

In [7]:
categories=os.listdir(dataPath)
categories

['Ripe Tomato', 'Green tomato', 'Rotten Tomatoes', 'Half ripe tomato']

In [8]:
vg19=VGG19(include_top=False,weights='imagenet',input_shape=imgShape)
vg19.output

80134624/80134624 [==============================] - 0s 0us/step


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [9]:
for layer in vg19.layers:
  layer.trainable=False

In [10]:
dataGen=ImageDataGenerator(rescale=1./255,
                           preprocessing_function=preprocess_input,
                           rotation_range=30,
                           shear_range=0.2,
                           zoom_range=0.1,
                           horizontal_flip=True,
                           fill_mode='nearest',
                           validation_split=0.2)

trainData=dataGen.flow_from_directory(dataPath,
                                      target_size=(imgShape[0],imgShape[1]),
                                      color_mode='rgb',
                                      class_mode='categorical',
                                      batch_size=batchSize,
                                      shuffle=True,
                                      subset='training')

valData=dataGen.flow_from_directory(dataPath,
                                      target_size=(imgShape[0],imgShape[1]),
                                      color_mode='rgb',
                                      class_mode='categorical',
                                      batch_size=batchSize,
                                      shuffle=True,
                                      subset='validation')

print(trainData.samples,valData.samples)

Found 3179 images belonging to 4 classes.
Found 792 images belonging to 4 classes.
3179 792


In [11]:
img,label=trainData.__next__()

In [12]:
x=Flatten()(vg19.output)
prediction=Dense(len(categories),activation='softmax')(x)
model=Model(inputs=vg19.inputs,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
!mkdir Models

In [14]:
numEpochs=50

In [15]:
filepath='Models/{epoch}-{val_accuracy:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath,monitor='val_accuracy',mode='max',save_best_only=True,verbose=1)
earlyStop=EarlyStopping(monitor='val_loss',patience=5,verbose=1)
reduceLR=ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=3,verbose=1,mode='max',min_lr=1e-10)
callbackList=[checkpoint,reduceLR,earlyStop]

In [16]:
optim=Adam(learning_rate=1e-4)
model.compile(optimizer = optim, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
modelHistory=model.fit(trainData,
                       steps_per_epoch=trainData.samples//batchSize,
                       validation_data=valData,
                       validation_steps=valData.samples//batchSize,
                       epochs=numEpochs,
                       callbacks=callbackList)

Epoch 1/50
198/198 [==============================] - ETA: 0s - loss: 0.1406 - accuracy: 0.9488
Epoch 1: val_accuracy did not improve from 0.87755
198/198 [==============================] - 58s 295ms/step - loss: 0.1406 - accuracy: 0.9488 - val_loss: 0.3659 - val_accuracy: 0.8431 - lr: 1.0000e-06
Epoch 2/50
198/198 [==============================] - ETA: 0s - loss: 0.1424 - accuracy: 0.9485
Epoch 2: val_accuracy did not improve from 0.87755
198/198 [==============================] - 58s 293ms/step - loss: 0.1424 - accuracy: 0.9485 - val_loss: 0.3593 - val_accuracy: 0.8533 - lr: 1.0000e-06
Epoch 3/50
198/198 [==============================] - ETA: 0s - loss: 0.1404 - accuracy: 0.9504
Epoch 3: val_accuracy did not improve from 0.87755
198/198 [==============================] - 58s 292ms/step - loss: 0.1404 - accuracy: 0.9504 - val_loss: 0.3551 - val_accuracy: 0.8610 - lr: 1.0000e-06
Epoch 4/50
198/198 [==============================] - ETA: 0s - loss: 0.1406 - accuracy: 0.9453
Epoch 4: v